<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [29]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_codigo.csv')



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [30]:
# --- Asegura nombres de columnas consistentes (sin tildes) ---
def normaliza_cols(df):
    df.columns = [c.strip().lower() for c in df.columns]
    ren = {'país': 'pais', 'año': 'anio', 'índice': 'indice'}  # posibles tildes
    df.rename(columns={k: v for k, v in ren.items() if k in df.columns}, inplace=True)
    return df

# a) Consolidación por años
dfs = []
for url in archivos_anio:
    t = pd.read_csv(url)
    normaliza_cols(t)
    dfs.append(t)
df_anio = pd.concat(dfs, ignore_index=True)


In [31]:
# Asegura minúsculas en df_codigos
df_codigos.columns = [c.strip().lower() for c in df_codigos.columns]

# (b) Si hay un ISO con dos países, deja solo uno.
#    — Minimalista: quedarte con la primera aparición por ISO.
dupes = df_codigos["codigo_iso"].duplicated(keep=False)
# opcional para que veas cuáles son los conflictivos:
# print(df_codigos.loc[dupes].sort_values("codigo_iso"))
df_codigos_clean = df_codigos.drop_duplicates(subset=["codigo_iso"], keep="first").copy()


In [32]:
# (c) Unión SOLO por coincidencias usando codigo_iso (inner)
df = pd.merge(
    df_anio,                               # ya trae codigo_iso, anio, indice, ranking
    df_codigos_clean[["codigo_iso","pais"]],
    on="codigo_iso",
    how="inner"
)

# Orden final de columnas como en el diccionario
df = df[["codigo_iso", "pais", "anio", "indice", "ranking"]]
print(df.columns)

Index(['codigo_iso', 'pais', 'anio', 'indice', 'ranking'], dtype='object')




### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [33]:
# ¿Cuántas filas (observaciones) contiene el conjunto de datos?
print("Filas:", df.shape[0])

Filas: 3060


In [34]:
# ¿Cuántas columnas tiene el DataFrame?
print("Columnas:", df.shape[1])


Columnas: 5


In [35]:
# ¿Cuáles son los nombres de las columnas?
print("Nombres de columnas:", df.columns.tolist())


Nombres de columnas: ['codigo_iso', 'pais', 'anio', 'indice', 'ranking']


In [36]:
# ¿Qué tipo de datos tiene cada columna?
print("\nTipos de datos por columna:")
print(df.dtypes)



Tipos de datos por columna:
codigo_iso     object
pais           object
anio            int64
indice        float64
ranking       float64
dtype: object


In [37]:
# ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?
# (Chequeo simple: esperamos 'codigo_iso' y 'pais' como objeto/string; 'anio' y 'ranking' numéricos; 'indice' numérico)
print("\nChequeo simple de tipos esperados:")
print(" - codigo_iso tipo texto?:", df["codigo_iso"].dtype == "object")
print(" - pais       tipo texto?:", df["pais"].dtype == "object")
print(" - anio       numérico?   :", pd.to_numeric(df["anio"], errors="coerce").notna().all())
print(" - ranking    numérico?   :", pd.to_numeric(df["ranking"], errors="coerce").notna().all())
print(" - indice     numérico?   :", pd.to_numeric(df["indice"], errors="coerce").notna().all())



Chequeo simple de tipos esperados:
 - codigo_iso tipo texto?: True
 - pais       tipo texto?: True
 - anio       numérico?   : True
 - ranking    numérico?   : False
 - indice     numérico?   : False


#### **Resumen Estadístico**

In [39]:
# Genera un resumen estadístico SOLO de columnas numéricas (compatible con pandas antiguo)
print("\nResumen estadístico (numéricos):")
num = df.select_dtypes(include=["number"])
print(num.describe())


Resumen estadístico (numéricos):
              anio        indice        ranking
count  3060.000000   2664.000000    2837.000000
mean   2009.941176    205.782316     477.930913
std       5.786024   2695.525264    6474.935347
min    2001.000000      0.000000       1.000000
25%    2005.000000     15.295000      34.000000
50%    2009.000000     28.000000      70.000000
75%    2015.000000     41.227500     110.000000
max    2019.000000  64536.000000  121056.000000


In [40]:
# ¿Qué valores mínimo, máximo y promedio tiene la columna 'indice'?
print("\nÍndice (min, max, mean):")
ind = pd.to_numeric(df["indice"], errors="coerce")  # robusto si viniera como texto
print(" - min:", ind.min())
print(" - max:", ind.max())
print(" - mean:", ind.mean())



Índice (min, max, mean):
 - min: 0.0
 - max: 64536.0
 - mean: 205.7823160660661


In [41]:
# ¿Qué países presentan los valores extremos en 'indice' y 'ranking'?
rk = pd.to_numeric(df["ranking"], errors="coerce")

paises_min_ind = df.loc[ind == ind.min(), "pais"].dropna().unique().tolist()
paises_max_ind = df.loc[ind == ind.max(), "pais"].dropna().unique().tolist()

paises_min_rk = df.loc[rk == rk.min(), "pais"].dropna().unique().tolist()
paises_max_rk = df.loc[rk == rk.max(), "pais"].dropna().unique().tolist()

print("\nPaís(es) con índice mínimo (mejor libertad):", paises_min_ind, " (indice =", ind.min(), ")")
print("País(es) con índice máximo (peor libertad):", paises_max_ind, " (indice =", ind.max(), ")")
print("País(es) con ranking mínimo (mejor puesto):", paises_min_rk, " (ranking =", rk.min(), ")")
print("País(es) con ranking máximo (peor puesto):", paises_max_rk, " (ranking =", rk.max(), ")")


País(es) con índice mínimo (mejor libertad): ['Dinamarca', 'Finlandia', 'Irlanda', 'Noruega', 'Suecia', 'Suiza', 'Islandia', 'Países Bajos']  (indice = 0.0 )
País(es) con índice máximo (peor libertad): ['Kosovo']  (indice = 64536.0 )
País(es) con ranking mínimo (mejor puesto): ['Finlandia', 'Islandia', 'Países Bajos', 'Noruega', 'Suiza', 'Dinamarca', 'Irlanda', 'Eslovaquia', 'Luxemburgo', 'Estonia', 'Austria', 'Nueva Zelanda']  (ranking = 1.0 )
País(es) con ranking máximo (peor puesto): ['Kosovo']  (ranking = 121056.0 )


####  **Datos faltantes**

In [48]:
#¿Cuántos valores nulos hay en cada columna?
nulos_por_col = df.isna().sum()
pct_nulos_por_col = (nulos_por_col / len(df) * 100) if len(df) else 0
resumen_nulos = (
    pd.DataFrame({
        "nulos": nulos_por_col,
        "pct_nulos": (nulos_por_col / len(df) * 100).round(2) if len(df) else nulos_por_col*0
    })
    .sort_values("nulos", ascending=False)
)
print("\nNulos por columna (ordenados):")
print(resumen_nulos.to_string())


Nulos por columna (ordenados):
            nulos  pct_nulos
indice        396      12.94
ranking       223       7.29
codigo_iso      0       0.00
anio            0       0.00
pais            0       0.00


In [49]:
#¿Qué proporción de observaciones tienen valores faltantes?
filas_con_nulo = df.isna().any(axis=1).sum()
prop_filas_con_nulo = (filas_con_nulo / len(df)) if len(df) else 0
print(f"\nFilas con ≥1 valor faltante: {filas_con_nulo} de {len(df)}  ({prop_filas_con_nulo:.2%})")



Filas con ≥1 valor faltante: 397 de 3060  (12.97%)


In [50]:
#¿Hay columnas con más del 30% de datos faltantes?
umbral = 0.30
cols_mas_30 = resumen_nulos[resumen_nulos["pct_nulos"] > umbral * 100]
if cols_mas_30.empty:
    print("\nNo hay columnas con >30% de nulos.")
else:
    print("\nColumnas con >30% de nulos:")
    print(cols_mas_30.to_string())


No hay columnas con >30% de nulos.


#### **Unicidad y duplicados**

In [51]:
#Cuántos países distintos (pais) hay en el DataFrame?
n_paises = df["pais"].nunique()
print("Países distintos:", n_paises)

Países distintos: 179


In [53]:
#¿Cuántos años distintos (anio) hay representados?
n_anios = df["anio"].nunique()
print("Años distintos:", n_anios)

Años distintos: 17


In [54]:
#¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?
n_dup_filas = df.duplicated(keep="first").sum()
print("Filas duplicadas (todas las columnas iguales):", n_dup_filas)

Filas duplicadas (todas las columnas iguales): 0


#### **Validación cruzada de columnas**

In [55]:
print("Validación cruzada de país y código ISO:")
display(df.groupby('codigo_iso')['pais'].nunique())


Validación cruzada de país y código ISO:


codigo_iso
AFG    1
AGO    1
ALB    1
AND    1
ARE    1
      ..
WSM    1
YEM    1
ZAF    1
ZMB    1
ZWE    1
Name: pais, Length: 180, dtype: int64




### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [61]:
# respuesta
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN']

df_america = df[df["codigo_iso"].isin(america)].copy()

#print(sorted(df_america["codigo_iso"].unique()))
#print(df_america.shape)

In [60]:
#a) 
anios = sorted(df_america["anio"].dropna().unique().tolist())
res_a = []
for a in anios:
    t = df_america[df_america["anio"] == a]
    if t.empty:
        continue
    min_val = t["indice"].min()
    max_val = t["indice"].max()
    pais_min = sorted(t.loc[t["indice"] == min_val, "pais"].dropna().unique().tolist())
    pais_max = sorted(t.loc[t["indice"] == max_val, "pais"].dropna().unique().tolist())
    res_a.append({
        "anio": int(a),
        "indice_min": float(min_val),
        "pais_min": ", ".join(pais_min),
        "indice_max": float(max_val),
        "pais_max": ", ".join(pais_max)
    })

extremos_for = pd.DataFrame(res_a).sort_values("anio")
print("\n(a) Extremos por año:")
print(extremos_for.to_string(index=False))



(a) Extremos por año:
 anio  indice_min          pais_min  indice_max  pais_max
 2001        0.80            Canadá       90.30      Cuba
 2002        1.00 Trinidad y Tobago       97.83      Cuba
 2003        2.00 Trinidad y Tobago    35826.00 Argentina
 2004        2.00 Trinidad y Tobago       87.00      Cuba
 2005        4.50   Bolivia, Canadá       95.00      Cuba
 2006        4.88            Canadá       96.17      Cuba
 2007        3.33            Canadá       88.33      Cuba
 2008        3.70            Canadá       94.00      Cuba
 2009        6.75    Estados Unidos       78.00      Cuba
 2012        9.88           Jamaica       71.64      Cuba
 2013       10.90           Jamaica       70.92      Cuba
 2014       10.99            Canadá       70.21      Cuba
 2015       11.10        Costa Rica       70.23      Cuba
 2016         NaN                           NaN          
 2017       11.93        Costa Rica       71.75      Cuba
 2018       11.33           Jamaica       68.90  

In [62]:
# (b) Enfoque vectorizado con groupby 
mins = df_america["indice"] == df_america.groupby("anio")["indice"].transform("min")
maxs = df_america["indice"] == df_america.groupby("anio")["indice"].transform("max")

# Tabla de mínimos por año (maneja empates)
min_tab = (
    df_america.loc[mins, ["anio", "indice", "pais"]]
      .groupby("anio")
      .agg({
          "indice": "min",
          "pais":   lambda x: ", ".join(sorted(pd.Series(x).dropna().unique().tolist()))
      })
      .rename(columns={"indice":"indice_min", "pais":"pais_min"})
      .reset_index()
)

# Tabla de máximos por año (maneja empates)
max_tab = (
    df_america.loc[maxs, ["anio", "indice", "pais"]]
      .groupby("anio")
      .agg({
          "indice": "max",
          "pais":   lambda x: ", ".join(sorted(pd.Series(x).dropna().unique().tolist()))
      })
      .rename(columns={"indice":"indice_max", "pais":"pais_max"})
      .reset_index()
)

extremos_gb = (
    min_tab.merge(max_tab, on="anio", how="inner")
           .sort_values("anio")
)

print("\n(b) Extremos por año (groupby vectorizado):")
print(extremos_gb.to_string(index=False))


(b) Extremos por año (groupby vectorizado):
 anio  indice_min          pais_min  indice_max  pais_max
 2001        0.80            Canadá       90.30      Cuba
 2002        1.00 Trinidad y Tobago       97.83      Cuba
 2003        2.00 Trinidad y Tobago    35826.00 Argentina
 2004        2.00 Trinidad y Tobago       87.00      Cuba
 2005        4.50   Bolivia, Canadá       95.00      Cuba
 2006        4.88            Canadá       96.17      Cuba
 2007        3.33            Canadá       88.33      Cuba
 2008        3.70            Canadá       94.00      Cuba
 2009        6.75    Estados Unidos       78.00      Cuba
 2012        9.88           Jamaica       71.64      Cuba
 2013       10.90           Jamaica       70.92      Cuba
 2014       10.99            Canadá       70.21      Cuba
 2015       11.10        Costa Rica       70.23      Cuba
 2017       11.93        Costa Rica       71.75      Cuba
 2018       11.33           Jamaica       68.90      Cuba
 2019       11.13          

### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [64]:
# Tarea principal:
df["indice"] = pd.to_numeric(df["indice"], errors="coerce")

pivot_indice_max = pd.pivot_table(
    df,
    index="pais",
    columns="anio",
    values="indice",
    aggfunc="max",
    fill_value=0
)

# (opcional) vista rápida
print("Dimensión (filas x columnas):", pivot_indice_max.shape)
print(pivot_indice_max.head().to_string())

Dimensión (filas x columnas): (179, 16)
anio         2001   2002   2003   2004   2005   2006   2007   2008   2009   2012   2013   2014   2015   2017   2018   2019
pais                                                                                                                      
Afghanistán  35.5  40.17  28.25  39.17  44.25  56.50  59.25  54.25  51.67  37.36  37.07  37.44  37.75  39.46  37.28  36.55
Albania       0.0   6.50  11.50  14.17  18.00  25.50  16.00  21.75  21.50  30.88  29.92  28.77  29.92  29.92  29.49  29.84
Alemania      1.5   1.33   2.00   4.00   5.50   5.75   4.50   3.50   4.25  10.24  10.23  11.47  14.80  14.97  14.39  14.60
Algeria      31.0  33.00  43.50  40.33  40.00  40.50  31.33  49.56  47.33  36.54  36.26  36.63  41.69  42.83  43.13  45.75
Andorra       0.0   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   6.82   6.82  19.87  19.87  21.03  22.21  24.63


#### **Preguntas adicionales**

In [66]:
# a)

max_indice_total = pivot_indice_max.max().max()
min_indice_total = pivot_indice_max[pivot_indice_max > 0].min().min()  # excluye ceros

pais_max_total = pivot_indice_max[pivot_indice_max == max_indice_total].stack().idxmax()[0]
pais_min_total = pivot_indice_max[pivot_indice_max == min_indice_total].stack().idxmin()[0]

print(f"País con el mayor índice en toda la tabla: {pais_max_total} ({max_indice_total})")
print(f"País con el menor índice (≠ 0) en toda la tabla: {pais_min_total} ({min_indice_total})")

País con el mayor índice en toda la tabla: Kosovo (64536.0)
País con el menor índice (≠ 0) en toda la tabla: Austria (0.5)


In [67]:
# b) ¿Qué años presentan en promedio los valores de indice más altos y más bajos?
prom_por_anio = pivot_indice_max.mean(axis=0)
valor_prom_mas_alto = prom_por_anio.max()
valor_prom_mas_bajo  = prom_por_anio.min()
anios_prom_mas_alto  = prom_por_anio[prom_por_anio == valor_prom_mas_alto].index.tolist()
anios_prom_mas_bajo  = prom_por_anio[prom_por_anio == valor_prom_mas_bajo].index.tolist()

print(f"Año(s) con el promedio de índice más alto: {anios_prom_mas_alto} ({valor_prom_mas_alto})")
print(f"Año(s) con el promedio de índice más bajo: {anios_prom_mas_bajo} ({valor_prom_mas_bajo})")


Año(s) con el promedio de índice más alto: [2013] (449.11446927374294)
Año(s) con el promedio de índice más bajo: [2001] (20.03240223463687)


In [68]:
# c) ¿Qué país muestra mayor variabilidad (max - min a lo largo del tiempo)?
variab = pivot_indice_max.max(axis=1) - pivot_indice_max.min(axis=1)
valor_variab_max = variab.max()
pais_mayor_variab = variab.idxmax()

print(f"País con la mayor variabilidad en el índice: {pais_mayor_variab} ({valor_variab_max})")


País con la mayor variabilidad en el índice: Kosovo (64536.0)


In [69]:
# d) ¿Existen países con índice constante en todos los años (excluyendo los que solo tienen ceros)?
mask_constante = (pivot_indice_max.max(axis=1) == pivot_indice_max.min(axis=1)) & (pivot_indice_max.min(axis=1) > 0)
constantes = pivot_indice_max.loc[mask_constante]

print("Países con índice constante en todos los años (excluyendo filas solo-ceros):")
display(constantes)

Países con índice constante en todos los años (excluyendo filas solo-ceros):


anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,


In [71]:
# e) ¿Qué países no tienen ningún dato (toda la fila = 0)? ¿Por qué?
mask_todo_cero = pivot_indice_max.sum(axis=1) == 0
sin_datos = pivot_indice_max.loc[mask_todo_cero]

print("Países sin datos (toda la fila igual a 0):")
display(sin_datos)


Países sin datos (toda la fila igual a 0):


anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,


## Explicación:
Estos países no registran observaciones en los archivos de origen para los años considerados; al crear la tabla dinámica se usó fill_value=0, por eso quedaron en 0 en todos los años.")